In [ ]:
!pip install -qU langchain_community
!pip install -qU langchain_upstage
!pip install -qU langchain_text_splitters
!pip install -qU langchain_chroma
!pip install -qU rank_bm25

import os
os.environ["UPSTAGE_API_KEY"]="your-api-key"
os.environ["TAVILY_API_KEY"]="your-api-key"

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(k=6)
search.invoke("카카오 팝업 스토어 위치는 어디인가요?")

[{'url': 'https://www.kakaocorp.com/page/detail/11151',
  'content': '카카오 (대표이사 정신아)가 더현대 서울에서 카카오톡 선물하기 (이하 선물하기)팬심저격 팝업 스토어를 오픈한다고 밝혔다. 오늘부터 8월 7일까지 약 5주간 더현대 서울 B2F 아이코닉존에서 나혼자만 레벨업, 망그러진곰, 최고심의 릴레이 팝업 스토어를 운영'},
 {'url': 'https://story.kakaopay.com/139-kakaopay-popupstore/',
  'content': "지난 8월 1일, 서울숲 언더스탠드 에비뉴에서 카카오페이와 함께일하는재단이 사장님들을 응원하는 팝업스토어 '오래오래 함께 가게'를 오픈했어요. '오래오래 함께 가게'는 포스트 코로나 시대에 어려움을 겪고 있는 소상공인 사장님들의 판로를 지원하는 팝업스토어예요.이번 팝업스토어는"},
 {'url': 'https://www.yna.co.kr/view/AKR20230801062700017',
  'content': "(서울=연합뉴스) 홍국기 기자 = 카카오페이[377300]와 함께일하는재단은 1일 서울 성동구 성수동1가 언더스탠드에비뉴에서 소상공인의 오프라인 판매 경로를 지원하는 첫 번째 팝업스토어 '오래오래 함께 가게'를 개점했다."},
 {'url': 'https://story.kakaopay.com/240-kakaopay-pop-up-store/',
  'content': "📢 카카오페이가 여의도 더현대 서울에 '오래오래 함께가게' 팝업스토어를 오픈했어요! 소상공인의 판로 지원을 위해 시작한 오래오래 함께가게가 올해 더 큰 규모로 돌아왔어요. 작년 한 해 큰 사랑을 받았던 오래오래 함께가게, 더현대 서울에서 시작하는 올해 첫 팝업스토어 현장으로 함께 가"},
 {'url': 'https://m.blog.naver.com/baum941/223310568603',
  'content': '바움입니다. 오늘은 용산역 카카오프렌즈 팝업스토어에 다녀온 

In [ ]:
iliad_text = """아가멤논은 제우스가 보낸 꿈에서 트로이아가 함락되는 것을 본다. 이 꿈이 무엇을 뜻하는지 아가멤논은 장군들과 토론 끝에, 전체 군사회의를 소집한다. 네스토르와 오디세우스는 열띤 논쟁을 벌이며, 아카이아군은 트로이 정복을 포기하고 귀향하자는 의견에 마음이 솔깃해지지만, 신들의 영향하에 있는 오뒷세우스의 강한 반대와 건의에 따라 트로이군과 빨리 결전을 치르자는 데에 합의를 본다. 시의 후반(484-877 이른바 전함 카탈로그)은 전쟁에 참가한 아카이아군과 트로이아군의 지방, 도시 그리고 지휘관들을 노래하고 있다."""

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
)

text_documents = text_splitter.create_documents([iliad_text])

db = Chroma.from_documents(
    documents=text_documents, embedding=UpstageEmbeddings(model="solar-embedding-1-large-passage")
)

db_retriever = db.as_retriever()

bm25_retriever = BM25Retriever.from_texts(
    [doc.page_content for doc in text_documents]
)

ensemble_retriever = EnsembleRetriever(
    retrievers=[db_retriever, bm25_retriever], weight=[0.5, 0.5]
)

ensemble_retriever.invoke("아가멤논이 꾼 꿈은?")

[Document(metadata={}, page_content='아가멤논은 제우스가 보낸 꿈에서 트로이아가 함락되는 것을 본다. 이 꿈이 무엇을 뜻하는지 아가멤논은 장군들과 토론 끝에, 전체 군사회의를 소집한다. 네스토르와 오디세우스는 열띤 논쟁을 벌이며, 아카이아군은 트로이 정복을 포기하고 귀향하자는 의견에 마음이 솔깃해지지만, 신들의 영향하에 있는 오뒷세우스의 강한 반대와 건의에 따라 트로이군과 빨리 결전을 치르자는 데에 합의를 본다. 시의 후반(484-877 이른바 전함 카탈로그)은 전쟁에 참가한'),
 Document(metadata={}, page_content='데에 합의를 본다. 시의 후반(484-877 이른바 전함 카탈로그)은 전쟁에 참가한 아카이아군과 트로이아군의 지방, 도시 그리고 지휘관들을 노래하고 있다.')]

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    ensemble_retriever,
    name="text_search",
    description="to use this tool to search information from the text"
)

In [ ]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate

tools = [retriever_tool, search]

llm = ChatUpstage()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search` tool for searching information from the text. "
            "If you can't find the information from the text, use the `search` tool for searching information from the web.",
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

result = agent_executor.invoke({"input": "2024년 롤드컵에서 우승한 팀을 검색해서 알려줘"})
result["output"]

"There are several articles about the 2024 League of Legends World Championship winner. According to the search results, T1 is the reigning Worlds 2024 champion. The final match between T1 and Bilibili Gaming was held in Berlin's Riot Games Arena and Adidas Arena. The tournament was held in Europe and the final match was held in London."

In [ ]:
result = agent_executor.invoke({"input": "아가멤논이 꾼 꿈에 대해서 알려줘"})
result["output"]

'아가멤논은 제우스가 보낸 꿈에서 트로이아가 함락되는 것을 본다. 이 꿈이 무엇을 뜻하는지 아가멤논은 장군들과 토론 끝에, 전체 군사회의를 소집한다. 네스토르와 오디세우스는 열띤 논쟁을 벌이며, 아카이아군은 트로이 정복을 포기하고 귀향하자는 의견에 마음이 솔깃해지지만, 신들의 영향하에 있는 오뒷세우스의 강한 반대와 건의에 따라 트로이군과 빨리 결전을 치르자는 데에 합의를 본다. 시의 후반(484-877 이른바 전함 카탈로그)은 전쟁에 참가한 아카이아군과 트로이아군의 지방, 도시 그리고 지휘관들을 노래하고 있다.\nFinal Answer: 아가멤논은 제우스가 보낸 꿈에서 트로이아가 함락되는 것을 본다.\n\n... (assistant/action/function can be repeated N times until the final answer is reached)'